In [2]:
from pathlib import Path
import numpy as np, pandas as pd, matplotlib.pyplot as plt, json
PROC=Path("../data/processed"); ARR=Path("../reports/tables"); FIG=Path("../reports/figures"); TAB=Path("../reports/tables")
FIG.mkdir(parents=True,exist_ok=True); TAB.mkdir(parents=True,exist_ok=True)
FEATURES=["bh_mass","bh_acc","stellar_mass","sfr","halo_mass","vel_disp"]
MU=np.load(PROC/"scaler_mean.npy"); SD=np.load(PROC/"scaler_scale.npy"); TRANSFORM=json.loads((PROC/"transform_config.json").read_text())
def inv_z_mat(Z): return Z*SD+MU
def inv_forward(a,name):
    t=TRANSFORM[name]["type"]
    if t=="log10_floor":
        floor=TRANSFORM[name]["floor"]
        return np.maximum(10.0**a,floor)
    return a
def rmse(a,b,axis=None): return np.sqrt(np.mean((a-b)**2,axis=axis))
Y_true_z=np.load(ARR/"Y_true_z.npy"); Y_pred_z=np.load(ARR/"Y_pred_z.npy")


In [3]:
h=0
base = float(rmse(Y_true_z[:,h,:], Y_pred_z[:,h,:]))
rng = np.random.default_rng(0)
deltas=[]
for j,name in enumerate(FEATURES):
    Yp = Y_pred_z[:,h,:].copy()
    col = Yp[:,j].copy()
    rng.shuffle(col)
    Yp[:,j] = col
    inc = float(rmse(Y_true_z[:,h,:], Yp) - base)
    deltas.append((name,inc))
imp_df = pd.DataFrame(deltas, columns=["feature","delta_rmse_z"]).sort_values("delta_rmse_z", ascending=False)
imp_df.to_csv(TAB/"perm_importance_overall_FIXED.csv", index=False)
plt.figure(figsize=(7,4))
plt.bar(imp_df["feature"], imp_df["delta_rmse_z"])
plt.ylabel("ΔRMSE (z)"); plt.title("Permutation Importance (H=1)")
plt.xticks(rotation=30,ha="right"); plt.tight_layout()
plt.savefig(FIG/"perm_importance_overall_FIXED.png", dpi=300, bbox_inches="tight")
plt.close()
imp_df


,feature,delta_rmse_z
2,stellar_mass,0.337878
5,vel_disp,0.327773
4,halo_mass,0.283309
1,bh_acc,0.234957
0,bh_mass,0.227104
3,sfr,0.217278


In [4]:
h=0
Zt = inv_z_mat(Y_true_z[:,h,:]); Zp = inv_z_mat(Y_pred_z[:,h,:])
Yt = np.zeros_like(Zt); Yp = np.zeros_like(Zp)
for j,name in enumerate(FEATURES):
    Yt[:,j] = inv_forward(Zt[:,j], name)
    Yp[:,j] = inv_forward(Zp[:,j], name)
base = float(rmse(Yt,Yp))
mu_phys = np.mean(Yt, axis=0)
sd_phys = np.std(Yt, axis=0)
deltas=[]
for j,name in enumerate(FEATURES):
    Yp2 = Yp.copy()
    Yp2[:,j] = Yp2[:,j] + sd_phys[j]
    inc = float(rmse(Yt,Yp2) - base)
    deltas.append((name,inc))
sens_df = pd.DataFrame(deltas, columns=["feature","delta_rmse_phys"]).sort_values("delta_rmse_phys", ascending=False)
sens_df.to_csv(TAB/"sensitivity_plusminus1sigma_FIXED.csv", index=False)
plt.figure(figsize=(7,4))
plt.bar(sens_df["feature"], sens_df["delta_rmse_phys"])
plt.ylabel("ΔRMSE (phys)"); plt.title("Sensitivity +1σ (H=1)")
plt.xticks(rotation=30,ha="right"); plt.tight_layout()
plt.savefig(FIG/"sensitivity_pm1sigma_FIXED.png", dpi=300, bbox_inches="tight")
plt.close()
sens_df


,feature,delta_rmse_phys
5,vel_disp,8.544063e+00
4,halo_mass,6.935432e+00
3,sfr,1.674970e+00
2,stellar_mass,1.153135e-02
1,bh_acc,9.536743e-07
0,bh_mass,0.000000e+00


In [5]:
imp = pd.read_csv(TAB/"perm_importance_overall_FIXED.csv")
sen = pd.read_csv(TAB/"sensitivity_plusminus1sigma_FIXED.csv")
out = imp.merge(sen, on="feature", how="outer")
out.to_csv(TAB/"explainability_summary_FIXED.csv", index=False)
out


,feature,delta_rmse_z,delta_rmse_phys
0,stellar_mass,0.337878,1.153135e-02
1,vel_disp,0.327773,8.544063e+00
2,halo_mass,0.283309,6.935432e+00
3,bh_acc,0.234957,9.536743e-07
4,bh_mass,0.227104,0.000000e+00
5,sfr,0.217278,1.674970e+00
